Prompt Optimization

Prompt Engineering recommends many best practices to build a prompt that completes the best output for the targetted goal.
It is an iterative process.

Setup environment

In [56]:
%pip install --upgrade IPython

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


This notebook started from codes in <a href="https://anthropic.skilljar.com/claude-with-the-anthropic-api/" target="_blank">Anthrophic Building with the Claude API</a> course.

The first class definition is to abstract away available LLM solutions.

In [6]:
import json
import concurrent.futures
import re
from textwrap import dedent
from statistics import mean
from dotenv import load_dotenv
from anthropic import Anthropic

class LLMClient:
    model = None

    def __init__(self):
        model=None

    def getModel(self):
        return model

In [15]:
from anthropic import Anthropic

class AntrophicClient(LLMClient):

    def __init__(self, model):
        LLMClient.__init__(self)
        self.model = model
        self.client = Anthropic()


    def add_user_message(self, messages, text):
        user_message = {"role": "user", "content": text}
        messages.append(user_message)


    def add_assistant_message(self, messages, text):
        assistant_message = {"role": "assistant", "content": text}
        messages.append(assistant_message)


    def chat(self, messages, system=None, temperature=1.0, stop_sequences=[]):
        params = {
            "model": model,
            "max_tokens": 1000,
            "messages": messages,
            "temperature": temperature,
            "stop_sequences": stop_sequences,
        }

        if system:
            params["system"] = system

        message = self.client.messages.create(**params)
        return message.content[0].text

TestSuiteGenerator is responsible for generating test prompts.


In [23]:
class TestSuiteGenerator():

    def __init__(self, client, max_concurrent_tasks=3):
        self.client=client
        self.max_concurrent_tasks=max_concurrent_tasks

    def render(self, template_string, variables):
        placeholders = re.findall(r"{([^{}]+)}", template_string)

        result = template_string
        for placeholder in placeholders:
            if placeholder in variables:
                result = result.replace(
                    "{" + placeholder + "}", str(variables[placeholder])
                )

        return result.replace("{{", "{").replace("}}", "}")

    def generate_unique_ideas(
        self, task_description, prompt_inputs_spec, num_cases
    ):
        """Generate a list of unique ideas for test cases based on the task description"""

        prompt = """
        Generate {num_cases} unique, diverse ideas for testing a prompt that accomplishes this task:

        <task_description>
        {task_description}
        </task_description>

        The prompt will receive the following inputs
        <prompt_inputs>
        {prompt_inputs_spec}
        </prompt_inputs>

        Each idea should represent a distinct scenario or example that tests different aspects of the task.

        Output Format:
        Provide your response as a structured JSON array where each item is a brief description of the idea.

        Example:
        ```json
        [
            "Testing with technical computer science terminology",
            "Testing with medical research findings",
            "Testing with complex mathematical concepts",
            ...
        ]
        ```

        Ensure each idea is:
        - Clearly distinct from the others
        - Relevant to the task description
        - Specific enough to guide generation of a full test case
        - Quick to solve without requiring extensive computation or multi-step processing
        - Solvable with no more than 400 tokens of output

        Remember, only generate {num_cases} unique ideas
        """

        system_prompt = "You are a test scenario designer specialized in creating diverse, unique testing scenarios."

        example_prompt_inputs = ""
        for key, value in prompt_inputs_spec.items():
            val = value.replace("\n", "\\n")
            example_prompt_inputs += f'"{key}": str # {val},'

        rendered_prompt = self.render(
            dedent(prompt),
            {
                "task_description": task_description,
                "num_cases": num_cases,
                "prompt_inputs": example_prompt_inputs,
            },
        )

        messages = []
        self.client.add_user_message(messages, rendered_prompt)
        self.client.add_assistant_message(messages, "```json")
        text = self.client.chat(
            messages,
            stop_sequences=["```"],
            system=system_prompt,
            temperature=1.0,
        )

        return json.loads(text)

    def generate_test_case(self, task_description, idea, prompt_inputs_spec={}):
        """Generate a single test case based on the task description and a specific idea"""

        example_prompt_inputs = ""
        for key, value in prompt_inputs_spec.items():
            val = value.replace("\n", "\\n")
            example_prompt_inputs += f'"{key}": "EXAMPLE_VALUE", // {val}\n'

        allowed_keys = ", ".join(
            [f'"{key}"' for key in prompt_inputs_spec.keys()]
        )

        prompt = """
        Generate a single detailed test case for a prompt evaluation based on:

        <task_description>
        {task_description}
        </task_description>

        <specific_idea>
        {idea}
        </specific_idea>

        <allowed_input_keys>
        {allowed_keys}
        </allowed_input_keys>

        Output Format:
        ```json
        {{
            "prompt_inputs": {{
            {example_prompt_inputs}
            }},
            "solution_criteria": ["criterion 1", "criterion 2", ...] // Concise list of criteria for evaluating the solution, 1 to 4 items
        }}
        ```

        IMPORTANT REQUIREMENTS:
        - You MUST ONLY use these exact input keys in your prompt_inputs: {allowed_keys}
        - Do NOT add any additional keys to prompt_inputs
        - All keys listed in allowed_input_keys must be included in your response
        - Make the test case realistic and practically useful
        - Include measurable, concise solution criteria
        - The solution criteria should ONLY address the direct requirements of the task description and the generated prompt_inputs
        - Avoid over-specifying criteria with requirements that go beyond the core task
        - Keep solution criteria simple, focused, and directly tied to the fundamental task
        - The test case should be tailored to the specific idea provided
        - Quick to solve without requiring extensive computation or multi-step processing
        - Solvable with no more than 400 tokens of output
        - DO NOT include any fields beyond those specified in the output format

        Here's an example of a sample input with an ideal output:
        <sample_input>
        <sample_task_description>
        Extract topics out of a passage of text
        </sample_task_description>
        <sample_specific_idea>
        Testing with a text that contains multiple nested topics and subtopics (e.g., a passage about renewable energy that covers solar power economics, wind turbine technology, and policy implications simultaneously)
        </sample_specific_idea>

        <sample_allowed_input_keys>
        "content"
        </sample_allowed_input_keys>
        </sample_input>
        <ideal_output>
        ```json
        {
            "prompt_inputs": {
                "content": "The transition to renewable energy encompasses numerous interdependent dimensions. Solar photovoltaic technology has seen dramatic cost reductions, with panel efficiency improving 24% since 2010 while manufacturing costs declined by 89%, making it economically competitive with fossil fuels in many markets. Concurrently, wind energy has evolved through innovative turbine designs featuring carbon-fiber composite blades and advanced control systems that increase energy capture by 35% in low-wind conditions."
            },
            "solution_criteria": [
                "Includes all topics mentioned"
            ]
        }
        ```
        </ideal_output>
        This is ideal output because the solution criteria is concise and doesn't ask for anything outside of the scope of the task description.
        """

        system_prompt = "You are a test case creator specializing in designing evaluation scenarios."

        rendered_prompt = self.render(
            dedent(prompt),
            {
                "allowed_keys": allowed_keys,
                "task_description": task_description,
                "idea": idea,
                "example_prompt_inputs": example_prompt_inputs,
            },
        )

        messages = []
        self.client.add_user_message(messages, rendered_prompt)
        self.client.add_assistant_message(messages, "```json")
        text = self.client.chat(
            messages,
            stop_sequences=["```"],
            system=system_prompt,
            temperature=0.7,
        )

        test_case = json.loads(text)
        test_case["task_description"] = task_description
        test_case["scenario"] = idea

        return test_case

    def generate_dataset(
        self,
        task_description,
        prompt_inputs_spec={},
        num_cases=1,
        output_file="dataset.json",
    ):
        """Generate test dataset based on task description and save to file"""
        ideas = self.generate_unique_ideas(
            task_description, prompt_inputs_spec, num_cases
        )

        dataset = []
        completed = 0
        total = len(ideas)
        last_reported_percentage = 0

        with concurrent.futures.ThreadPoolExecutor(
            max_workers=self.max_concurrent_tasks
        ) as executor:
            future_to_idea = {
                executor.submit(
                    self.generate_test_case,
                    task_description,
                    idea,
                    prompt_inputs_spec,
                ): idea
                for idea in ideas
            }

            for future in concurrent.futures.as_completed(future_to_idea):
                try:
                    result = future.result()
                    completed += 1
                    current_percentage = int((completed / total) * 100)
                    milestone_percentage = (current_percentage // 20) * 20

                    if milestone_percentage > last_reported_percentage:
                        print(f"Generated {completed}/{total} test cases")
                        last_reported_percentage = milestone_percentage

                    dataset.append(result)
                except Exception as e:
                    print(f"Error generating test case: {e}")

        with open(output_file, "w") as f:
            json.dump(dataset, f, indent=2)

        return dataset



In [37]:
class PromptOutputGrader():

    def __init__(self, client, max_concurrent_tasks=3):
        self.client=client
        self.max_concurrent_tasks=max_concurrent_tasks

    def generate_prompt_evaluation_report(self, evaluation_results):
        total_tests = len(evaluation_results)
        scores = [result["score"] for result in evaluation_results]
        avg_score = mean(scores) if scores else 0
        max_possible_score = 10
        pass_rate = (
            100 * len([s for s in scores if s >= 7]) / total_tests
            if total_tests
            else 0
        )

        html = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Prompt Evaluation Report</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
            }}
            .header {{
                background-color: #f0f0f0;
                padding: 20px;
                border-radius: 5px;
                margin-bottom: 20px;
            }}
            .summary-stats {{
                display: flex;
                justify-content: space-between;
                flex-wrap: wrap;
                gap: 10px;
            }}
            .stat-box {{
                background-color: #fff;
                border-radius: 5px;
                padding: 15px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                flex-basis: 30%;
                min-width: 200px;
            }}
            .stat-value {{
                font-size: 24px;
                font-weight: bold;
                margin-top: 5px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }}
            th {{
                background-color: #4a4a4a;
                color: white;
                text-align: left;
                padding: 12px;
            }}
            td {{
                padding: 10px;
                border-bottom: 1px solid #ddd;
                vertical-align: top;
            }}
            tr:nth-child(even) {{
                background-color: #f9f9f9;
            }}
            .output-cell {{
                white-space: pre-wrap;
            }}
            .score {{
                font-weight: bold;
                padding: 5px 10px;
                border-radius: 3px;
                display: inline-block;
            }}
            .score-high {{
                background-color: #c8e6c9;
                color: #2e7d32;
            }}
            .score-medium {{
                background-color: #fff9c4;
                color: #f57f17;
            }}
            .score-low {{
                background-color: #ffcdd2;
                color: #c62828;
            }}
            .output {{
                overflow: auto;
                white-space: pre-wrap;
            }}

            .output pre {{
                background-color: #f5f5f5;
                border: 1px solid #ddd;
                border-radius: 4px;
                padding: 10px;
                margin: 0;
                font-family: 'Consolas', 'Monaco', 'Courier New', monospace;
                font-size: 14px;
                line-height: 1.4;
                color: #333;
                box-shadow: inset 0 1px 3px rgba(0, 0, 0, 0.1);
                overflow-x: auto;
                white-space: pre-wrap;
                word-wrap: break-word;
            }}

            td {{
                width: 20%;
            }}
            .score-col {{
                width: 80px;
            }}
        </style>
        </head>
        <body>
        <div class="header">
            <h1>Prompt Evaluation Report</h1>
            <div class="summary-stats">
                <div class="stat-box">
                    <div>Total Test Cases</div>
                    <div class="stat-value">{total_tests}</div>
                </div>
                <div class="stat-box">
                    <div>Average Score</div>
                    <div class="stat-value">{avg_score:.1f} / {max_possible_score}</div>
                </div>
                <div class="stat-box">
                    <div>Pass Rate (≥7)</div>
                    <div class="stat-value">{pass_rate:.1f}%</div>
                </div>
            </div>
        </div>

        <table>
            <thead>
                <tr>
                    <th>Scenario</th>
                    <th>Prompt Inputs</th>
                    <th>Solution Criteria</th>
                    <th>Output</th>
                    <th>Score</th>
                    <th>Reasoning</th>
                </tr>
            </thead>
            <tbody>
        """

    def render(self, template_string, variables):
        placeholders = re.findall(r"{([^{}]+)}", template_string)

        result = template_string
        for placeholder in placeholders:
            if placeholder in variables:
                result = result.replace(
                    "{" + placeholder + "}", str(variables[placeholder])
                )

        return result.replace("{{", "{").replace("}}", "}")


    def grade_output(self, test_case, output, extra_criteria):
        """Grade the output of a test case using the model"""

        prompt_inputs = ""
        for key, value in test_case["prompt_inputs"].items():
            val = value.replace("\n", "\\n")
            prompt_inputs += f'"{key}":"{val}",\n'

        extra_criteria_section = ""
        if extra_criteria:
            extra_criteria_template = """
            Mandatory Requirements - ANY VIOLATION MEANS AUTOMATIC FAILURE (score of 3 or lower):
            <extra_important_criteria>
            {extra_criteria}
            </extra_important_criteria>
            """
            extra_criteria_section = self.render(
                dedent(extra_criteria_template),
                {"extra_criteria": extra_criteria},
            )

        eval_template = """
        Your task is to evaluate the following AI-generated solution with EXTREME RIGOR.

        Original task description:
        <task_description>
        {task_description}
        </task_description>

        Original task inputs:
        <task_inputs>
        {{ {prompt_inputs} }}
        </task_inputs>

        Solution to Evaluate:
        <solution>
        {output}
        </solution>

        Criteria you should use to evaluate the solution:
        <criteria>
        {solution_criteria}
        </criteria>

        {extra_criteria_section}

        Scoring Guidelines:
        * Score 1-3: Solution fails to meet one or more MANDATORY requirements
        * Score 4-6: Solution meets all mandatory requirements but has significant deficiencies in secondary criteria
        * Score 7-8: Solution meets all mandatory requirements and most secondary criteria, with minor issues
        * Score 9-10: Solution meets all mandatory and secondary criteria

        IMPORTANT SCORING INSTRUCTIONS:
        * Grade the output based ONLY on the listed criteria. Do not add your own extra requirements.
        * If a solution meets all of the mandatory and secondary criteria give it a 10
        * Don't complain that the solution "only" meets the mandatory and secondary criteria. Solutions shouldn't go above and beyond - they should meet the exact listed criteria.
        * ANY violation of a mandatory requirement MUST result in a score of 3 or lower
        * The full 1-10 scale should be utilized - don't hesitate to give low scores when warranted

        Output Format
        Provide your evaluation as a structured JSON object with the following fields, in this specific order:
        - "strengths": An array of 1-3 key strengths
        - "weaknesses": An array of 1-3 key areas for improvement
        - "reasoning": A concise explanation of your overall assessment
        - "score": A number between 1-10

        Respond with JSON. Keep your response concise and direct.
        Example response shape:
        {{
            "strengths": string[],
            "weaknesses": string[],
            "reasoning": string,
            "score": number
        }}
        """

        eval_prompt = self.render(
            dedent(eval_template),
            {
                "task_description": test_case["task_description"],
                "prompt_inputs": prompt_inputs,
                "output": output,
                "solution_criteria": "\n".join(test_case["solution_criteria"]),
                "extra_criteria_section": extra_criteria_section,
            },
        )

        messages = []
        self.client.add_user_message(messages, eval_prompt)
        self.client.add_assistant_message(messages, "```json")
        eval_text = self.client.chat(
            messages,
            stop_sequences=["```"],
            temperature=0.0,
        )
        return json.loads(eval_text)

    def run_test_case(
        self, test_case, run_prompt_function, extra_criteria=None
    ):
        """Run a test case and grade the result"""
        output = run_prompt_function(self.client,test_case["prompt_inputs"])

        model_grade = self.grade_output(test_case, output, extra_criteria)
        model_score = model_grade["score"]
        reasoning = model_grade["reasoning"]

        return {
            "output": output,
            "test_case": test_case,
            "score": model_score,
            "reasoning": reasoning,
        }

    def run_evaluation(
        self,
        run_prompt_function,
        dataset_file,
        extra_criteria=None,
        json_output_file="output.json",
        html_output_file="output.html",
    ):
        """Run evaluation on all test cases in the dataset"""
        with open(dataset_file, "r") as f:
            dataset = json.load(f)

        results = []
        completed = 0
        total = len(dataset)
        last_reported_percentage = 0

        with concurrent.futures.ThreadPoolExecutor(
            max_workers=self.max_concurrent_tasks
        ) as executor:
            future_to_test_case = {
                executor.submit(
                    self.run_test_case,
                    test_case,
                    run_prompt_function,
                    extra_criteria,
                ): test_case
                for test_case in dataset
            }

            for future in concurrent.futures.as_completed(future_to_test_case):
                result = future.result()
                completed += 1
                current_percentage = int((completed / total) * 100)
                milestone_percentage = (current_percentage // 20) * 20

                if milestone_percentage > last_reported_percentage:
                    print(f"Graded {completed}/{total} test cases")
                    last_reported_percentage = milestone_percentage
                results.append(result)

        average_score = mean([result["score"] for result in results])
        print(f"Average score: {average_score}")

        with open(json_output_file, "w") as f:
            json.dump(results, f, indent=2)

        #html = self.generate_prompt_evaluation_report(results)
        #with open(html_output_file, "w", encoding="utf-8") as f:
        #    f.write(html)

        return results


In [49]:
class EvaluationWriter():

    def __init__(self):
        pass

    def generate_prompt_evaluation_report(self, evaluation_results):
        total_tests = len(evaluation_results)
        scores = [result["score"] for result in evaluation_results]
        avg_score = mean(scores) if scores else 0
        max_possible_score = 10
        pass_rate = (
            100 * len([s for s in scores if s >= 7]) / total_tests
            if total_tests
            else 0
        )

        html = f"""
        <!DOCTYPE html>
        <html lang="en">
        <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Prompt Evaluation Report</title>
        <style>
            body {{
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
            }}
            .header {{
                background-color: #f0f0f0;
                padding: 20px;
                border-radius: 5px;
                margin-bottom: 20px;
            }}
            .summary-stats {{
                display: flex;
                justify-content: space-between;
                flex-wrap: wrap;
                gap: 10px;
            }}
            .stat-box {{
                background-color: #fff;
                border-radius: 5px;
                padding: 15px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
                flex-basis: 30%;
                min-width: 200px;
            }}
            .stat-value {{
                font-size: 24px;
                font-weight: bold;
                margin-top: 5px;
            }}
            table {{
                width: 100%;
                border-collapse: collapse;
                margin-top: 20px;
            }}
            th {{
                background-color: #4a4a4a;
                color: white;
                text-align: left;
                padding: 12px;
            }}
            td {{
                padding: 10px;
                border-bottom: 1px solid #ddd;
                vertical-align: top;
            }}
            tr:nth-child(even) {{
                background-color: #f9f9f9;
            }}
            .output-cell {{
                white-space: pre-wrap;
            }}
            .score {{
                font-weight: bold;
                padding: 5px 10px;
                border-radius: 3px;
                display: inline-block;
            }}
            .score-high {{
                background-color: #c8e6c9;
                color: #2e7d32;
            }}
            .score-medium {{
                background-color: #fff9c4;
                color: #f57f17;
            }}
            .score-low {{
                background-color: #ffcdd2;
                color: #c62828;
            }}
            .output {{
                overflow: auto;
                white-space: pre-wrap;
            }}

            .output pre {{
                background-color: #f5f5f5;
                border: 1px solid #ddd;
                border-radius: 4px;
                padding: 10px;
                margin: 0;
                font-family: 'Consolas', 'Monaco', 'Courier New', monospace;
                font-size: 14px;
                line-height: 1.4;
                color: #333;
                box-shadow: inset 0 1px 3px rgba(0, 0, 0, 0.1);
                overflow-x: auto;
                white-space: pre-wrap;
                word-wrap: break-word;
            }}

            td {{
                width: 20%;
            }}
            .score-col {{
                width: 80px;
            }}
        </style>
        </head>
        <body>
        <div class="header">
            <h1>Prompt Evaluation Report</h1>
            <div class="summary-stats">
                <div class="stat-box">
                    <div>Total Test Cases</div>
                    <div class="stat-value">{total_tests}</div>
                </div>
                <div class="stat-box">
                    <div>Average Score</div>
                    <div class="stat-value">{avg_score:.1f} / {max_possible_score}</div>
                </div>
                <div class="stat-box">
                    <div>Pass Rate (≥7)</div>
                    <div class="stat-value">{pass_rate:.1f}%</div>
                </div>
            </div>
        </div>

        <table>
            <thead>
                <tr>
                    <th>Scenario</th>
                    <th>Prompt Inputs</th>
                    <th>Solution Criteria</th>
                    <th>Output</th>
                    <th>Score</th>
                    <th>Reasoning</th>
                </tr>
            </thead>
            <tbody>
        """

        for result in evaluation_results:
            prompt_inputs_html = "<br>".join(
                [
                f"<strong>{key}:</strong> {value}"
                for key, value in result["test_case"]["prompt_inputs"].items()
                ]
            )

            criteria_string = "<br>• ".join(
                result["test_case"]["solution_criteria"]
            )

            score = result["score"]
            if score >= 8:
                score_class = "score-high"
            elif score <= 5:
                score_class = "score-low"
            else:
                score_class = "score-medium"

            html += f"""
                <tr>
                    <td>{result["test_case"]["scenario"]}</td>
                    <td class="prompt-inputs">{prompt_inputs_html}</td>
                    <td class="criteria">• {criteria_string}</td>
                    <td class="output"><pre>{result["output"]}</pre></td>
                    <td class="score-col"><span class="score {score_class}">{score}</span></td>
                    <td class="reasoning">{result["reasoning"]}</td>
                </tr>
                """

        html += """
                </tbody>
            </table>
        </body>
        </html>
        """

        return html

    def writeTo(self,results,html_output_file="eval_results.html"):
        html = self.generate_prompt_evaluation_report(results)
        print(html)
        with open(html_output_file, "w", encoding="utf-8") as f:
            f.write(html)



Ask LLM to generate test prompts


In [24]:
load_dotenv()

model = "claude-3-5-haiku-latest"
client = AntrophicClient(model)

# Create an instance of TestSuiteGenerator
# Pass LLMClient
# Set `max_concurrent_tasks`, may Increase `max_concurrent_tasks` for higher concurrency, may hit API rate limiting 429
evaluator =  TestSuiteGenerator(client, max_concurrent_tasks=1)


In [25]:
#
# What is the purpose / goal of the prompt under test (PUT)?
# What is the best way to specify PUT?
generator_task_description="Write a compact, concise 1 day meal plan for a single athlete"

# What are the inputs that prompt under test (PUT) requires
# Consider XML formatted additional input to PUT
generator_prompt_inputs_spec={
        "height": "Athlete's height in cm",
        "weight": "Athlete's weight in kg",
        "goal": "Goal of the athlete",
        "restrictions": "Dietary restrictions of the athlete",
    }

# Number of test cases to be generated
# Because the TestSuiteGenerator uses multiple threads to generate test cases faster,
# mindful of max_concurrent_tasks and generator_number_of_test_cases to avoid getting 429 from API call.
generator_number_of_test_cases=3

# Writes generated dataset to "./"+ generator_output_file_name
generator_output_file_name="dataset-v5.json"

# Ask TestSuiteGenerator to generate test data
dataset = evaluator.generate_dataset(
    task_description=generator_task_description,
    prompt_inputs_spec=generator_prompt_inputs_spec,
    output_file=generator_output_file_name,
    num_cases=generator_number_of_test_cases,
)

Generated 1/3 test cases
Generated 2/3 test cases
Generated 3/3 test cases


Let's see what the test suite consists of

In [26]:
print(json.dumps(dataset, indent=2))

[
  {
    "prompt_inputs": {
      "height": "180",
      "weight": "75",
      "goal": "Optimize endurance performance for marathon training",
      "restrictions": "Lactose intolerant, no red meat"
    },
    "solution_criteria": [
      "Meal plan meets 3000-3500 daily calorie requirement",
      "Macronutrient balance targets 55-60% carbohydrates, 20-25% protein, 20-25% healthy fats",
      "Provides nutrition supporting high-intensity endurance training",
      "Accommodates specified dietary restrictions"
    ],
    "task_description": "Write a compact, concise 1 day meal plan for a single athlete",
    "scenario": "High-intensity endurance athlete training diet with specific caloric and macro requirements"
  },
  {
    "prompt_inputs": {
      "height": "180",
      "weight": "82",
      "goal": "muscle maintenance and athletic performance during exam week",
      "restrictions": "limited cooking equipment, budget under $15/day, lactose intolerant"
    },
    "solution_criteria"

For different "scenario", it generated a different set of prompt_input array and "solution_criteria" to be validated.

In [31]:
# Define and run the prompt you want to evaluate, returning the raw model output
# This function is executed once for each test case
def run_prompt(client, prompt_inputs):
    prompt = f"""
    Generate a one-day meal plan for an athlete that meets their dietary restrictions.

    - Height : {prompt_inputs["height"]} cm
    - Weight : {prompt_inputs["weight"]} kg
    - Goal : {prompt_inputs["goal"]} kg
    - Dietary restrictions : {prompt_inputs["restrictions"]}

    Guidelines:
    1. Include accurate daily calorie amount
    2. Show protein, fat, and carb amounts
    3. Specify when to eat each meal
    4. Use only foods that fit restrictions
    5. List all portion sizes in grams
    6. Keep budget-friendly if mentioned
    """

    messages = []
    client.add_user_message(messages, prompt)
    return client.chat(messages)

In [42]:
# Create Evaluator
grader = PromptOutputGrader(client,max_concurrent_tasks=1)
# run_prompt_function function it sends prompt to LLM
# dataset_file
# extra_criteria
results = grader.run_evaluation(
    run_prompt_function=run_prompt,
    dataset_file="dataset-v5.json",
    extra_criteria="""
     The output should include:
    - Daily caloric total
    - Macronutrient breakdown
    - Meals with exact foods, portions, and timing
    """,
)

Graded 1/3 test cases
Graded 2/3 test cases
Graded 3/3 test cases
Average score: 6.333333333333333


In [43]:
print(json.dumps(results, indent=2))

[
  {
    "output": "Athlete Meal Plan for Marathon Training\n\nDaily Calorie Needs Calculation:\n- Basal Metabolic Rate (BMR): 1,800 calories\n- Training Volume: +600 calories\n- Total Daily Calories: 2,400 calories\n\nMacronutrient Breakdown:\n- Protein: 135g (22.5%) - 540 calories\n- Carbohydrates: 360g (60%) - 1,440 calories\n- Fats: 80g (30%) - 720 calories\n\nMeal Plan:\n\n\ud83c\udf73 Breakfast (7:00 AM) - 600 calories\n- Oatmeal with chia seeds (80g)\n- Banana (120g)\n- Almond butter (30g)\n- Plant-based protein shake (30g)\n- Protein: 35g, Carbs: 75g, Fats: 20g\n\n\ud83e\udd57 Mid-Morning Snack (10:00 AM) - 300 calories\n- Quinoa salad with grilled chicken (100g)\n- Mixed vegetables (100g)\n- Olive oil dressing (15g)\n- Protein: 25g, Carbs: 30g, Fats: 15g\n\n\ud83c\udf7d\ufe0f Lunch (1:00 PM) - 550 calories\n- Grilled salmon (120g)\n- Sweet potato (150g)\n- Steamed broccoli (100g)\n- Protein: 30g, Carbs: 60g, Fats: 20g\n\n\ud83c\udf4e Afternoon Snack (4:00 PM) - 250 calories\n

Let's write to HTML file

In [50]:
htmlWriter= EvaluationWriter()
htmlWriter.writeTo(results,html_output_file="evaluation_results.html")



        <!DOCTYPE html>
        <html lang="en">
        <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Prompt Evaluation Report</title>
        <style>
            body {
                font-family: Arial, sans-serif;
                line-height: 1.6;
                margin: 0;
                padding: 20px;
                color: #333;
            }
            .header {
                background-color: #f0f0f0;
                padding: 20px;
                border-radius: 5px;
                margin-bottom: 20px;
            }
            .summary-stats {
                display: flex;
                justify-content: space-between;
                flex-wrap: wrap;
                gap: 10px;
            }
            .stat-box {
                background-color: #fff;
                border-radius: 5px;
                padding: 15px;
                box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        

The following code renders the evaluation_results.html file, generated by the code in <a href="https://anthropic.skilljar.com/claude-with-the-anthropic-api/" target="_blank">Anthrophic Building with the Claude API</a>

In [62]:
from IPython.display import IFrame

IFrame(src='./evaluation_results.html', width=1200, height=600)

<h1>References</h1>
<ol>
<li> <a href="https://anthropic.skilljar.com/claude-with-the-anthropic-api/" target="_blank">Anthrophic Building with the Claude API</a>
</ol>